# Avnish Tripathi 
# 22111014
# Assingment 3

In [1]:
# importing necessary modules
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
import ipywidgets as widgets
from vtk import *

c:\Users\pc\miniconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\pc\miniconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\pc\miniconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# Create the dataset
reader = vtkXMLImageDataReader()
reader.SetFileName('mixture.vti')
reader.Update()
data = reader.GetOutput()

In [3]:
def get_num_points():
    ''' functions to return num_of_points'''
    num_dimensions = data.GetDimensions()
    num_points = num_dimensions[0] * num_dimensions[1] * num_dimensions[2]
    # print(num_points)
    return num_points

In [4]:
array_of_data = data.GetPointData().GetArray('ImageFile')
values = [array_of_data.GetTuple1(pid) for pid in range(get_num_points())]
Range = data.GetScalarRange()
# print(Range)

In [5]:
def get_coordinates():
    coordinates = [data.GetPoint(pid) for pid in range(get_num_points())]
    x = [i[0] for i in coordinates]
    y = [i[1] for i in coordinates]
    z = [i[2] for i in coordinates]
    return x , y , z

In [6]:
x , y , z = get_coordinates()

In [7]:
# Initial ISosurface
fig1 = go.FigureWidget(data=go.Isosurface(
    x = x,
    y = y,
    z = z,
    value = values,
    isomin = 0.0,
    isomax = 0.0,
    caps = dict(x_show=False, y_show=False),  #setting up the values of parameter
    colorscale = 'plasma',
    cauto = False,
    cmin = min(values),
    cmax = max(values),
    opacity = 1,
    colorbar = dict(thickness = 20,ticklen=4),
    showscale = False
))
# fig1.show()

In [8]:
# Create a histogram 
histoGram = go.Histogram(x = values,nbinsx = 30)

# Create a layout object
layout = go.Layout(
    xaxis = dict(title='Vortex Scalar Values'),
    yaxis = dict(title='Frequency'),
)

# Create a figure object
fig2 = go.FigureWidget(data=[histoGram], layout=layout)
# fig2.show()

In [9]:
# Create the widgets
slider = widgets.FloatSlider(value = 0.0, min = Range[0], max = Range[1], description = 'Isovalue',continuous_update = False,step = 0.01)
button = widgets.Button(description='Reset')

def update_values(isovalue):
    return [val for val in values if val >= isovalue - 0.25 and val <= isovalue + 0.25]

def update(change):
    isovalue = change.new
    fig1.data[0].isomin = isovalue
    fig1.data[0].isomax = isovalue
    # Updating the histogram data based on slider value
    fig2.data[0].x = update_values(isovalue)
    
    
def reset(button):
    # Reset isovalue to initial value
    slider.value = 0.0
    isovalue = 0.0
    fig1.data[0].update(isomin=isovalue, isomax=isovalue)
    # Reset histogram data
    fig2.data[0].x = values
    
# Link the slider to the update function
slider.observe(update, names='value')

# Link the button to the reset function
button.on_click(reset)

# Create the layout for the widget
Graph = widgets.VBox([
    widgets.HBox([slider, button]), 
    widgets.HBox([fig1, fig2])
])

# Display the widget
display(Graph)

##### 